In [13]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Importing and Cleaning Data

In [14]:
df = pd.read_csv(r"Z:\Data\RIDERSHIP\RidershipData.csv")
df.head()

,Route,Day,Ride Start,Stop,Scheduled Time,Day Of Week,Vehicle Capacity,Ride State,Stop State,Actual Arrival,Actual Departure,Riders On,Riders Off,Riders Left,Riders Cumulative
0,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:34:00,Thursday,45.0,Complete,Departed,2023-08-31 07:54:39,2023-08-31 07:54:52,0,0,0,0
1,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:35:00,Thursday,45.0,Complete,Departed,2023-08-31 07:55:27,2023-08-31 07:59:10,0,0,0,0
2,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River Apts,2023-08-31 07:37:00,Thursday,45.0,Complete,Departed,2023-08-31 08:01:02,2023-08-31 08:01:23,0,0,0,0
3,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:38:00,Thursday,45.0,Complete,Departed,2023-08-31 08:02:24,2023-08-31 08:02:34,0,0,0,0
4,Campus Shuttle,2023/08/31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:54:00,Thursday,45.0,Complete,Departed,2023-08-31 08:03:07,2023-08-31 08:04:03,0,0,0,0


In [15]:
# Filter rows where 'Route' contains the word 'Boston'
filtered_df = df[df['Stop'].str.contains('Crescent St')]

# Display the filtered DataFrame
print(filtered_df['Stop'])

319       Crescent St @ Cherry St (in front of Watch Fac...
320                                Crescent St and Brown St
321             Crescent St and Orange St (Crescent Suites)
324                               Crescent St and Walnut St
346       Crescent St @ Cherry St (in front of Watch Fac...
                                ...                        
160630                 Crescent St and Brown St (MBTA Stop)
160631    Crescent St and Orange St (Crescent Suites) (M...
160665    Crescent St @ Cherry St (in front of Watch Fac...
160666                 Crescent St and Brown St (MBTA Stop)
160667    Crescent St and Orange St (Crescent Suites) (M...
Name: Stop, Length: 11177, dtype: object


In [16]:
date_format = '%Y/%m/%d'
df['Day'] = pd.to_datetime(df['Day'], format=date_format)
df.head()

,Route,Day,Ride Start,Stop,Scheduled Time,Day Of Week,Vehicle Capacity,Ride State,Stop State,Actual Arrival,Actual Departure,Riders On,Riders Off,Riders Left,Riders Cumulative
0,Campus Shuttle,2023-08-31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:34:00,Thursday,45.0,Complete,Departed,2023-08-31 07:54:39,2023-08-31 07:54:52,0,0,0,0
1,Campus Shuttle,2023-08-31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:35:00,Thursday,45.0,Complete,Departed,2023-08-31 07:55:27,2023-08-31 07:59:10,0,0,0,0
2,Campus Shuttle,2023-08-31,2023-08-31 07:30:00,Charles River Apts,2023-08-31 07:37:00,Thursday,45.0,Complete,Departed,2023-08-31 08:01:02,2023-08-31 08:01:23,0,0,0,0
3,Campus Shuttle,2023-08-31,2023-08-31 07:30:00,Charles River parking lot,2023-08-31 07:38:00,Thursday,45.0,Complete,Departed,2023-08-31 08:02:24,2023-08-31 08:02:34,0,0,0,0
4,Campus Shuttle,2023-08-31,2023-08-31 07:30:00,South St and Turner Rd,2023-08-31 07:54:00,Thursday,45.0,Complete,Departed,2023-08-31 08:03:07,2023-08-31 08:04:03,0,0,0,0


In [17]:
# selected = 'Riders On'
selected = 'Riders Off'

print(df[selected])

0         0
1         0
2         0
3         0
4         0
         ..
160690    0
160691    0
160692    0
160693    0
160694    0
Name: Riders Off, Length: 160695, dtype: int64


# Making Dash App

In [18]:
app = dash.Dash(__name__)

## Layout

In [19]:
# Layout of the dashboard
app.layout = html.Div([
    # ROUTE SELECTOR
    html.Div([
        dcc.Dropdown(
            id='route-selector',
            options=[{'label': route, 'value': route} for route in df['Route'].unique()],
            value=df['Route'].unique()[0],
            style={'font-family': 'Segoe UI'}
        ),
        dcc.Dropdown(
            id='stop-selector',
            options=[],  # Initialize with an empty list
            value=[],
            multi=True,
            style={'max-height': '80px', 'overflow-y': 'auto','font-family': 'Segoe UI'}
        )
    ], style={'width': '40%', 'display': 'inline-block', 'padding-left': '2em'}),
    
    # RIDERS ON/OFF SELECTOR, DATE SELECTOR, and DAY OF WEEK SELECTOR
    html.Div([
        dcc.DatePickerRange(
            id='date-slider',
            display_format='YYYY-MM-DD',
            style={'font-family': 'Segoe UI'}
        )
    ], style={'width': '40%', 'display': 'inline-block', 'float': 'right', 'padding-left': '0em'}),

    # AVERAGE STOP RIDERSHIP GRAPH
    dcc.Graph(id='stop-bar-chart'),
    
    # RIDERS ON/OFF SELECTOR
    html.Div([
        dcc.Dropdown(
            id='riders-selector',
            options=[
                {'label': 'Riders On', 'value': 'Riders On'},
                {'label': 'Riders Off', 'value': 'Riders Off'}
            ],
            value='Riders On',
            style={'font-family': 'Segoe UI'}
        ),
    ], style={'width': '80%', 'display': 'inline-block', 'padding-left': '2em'}),

    # STOP-SPECIFIC AVERAGE RIDERSHIP
    # STOP DROPDOWN - Empty initially, to be populated based on route selection
    html.Div([
        html.Label("\nSelect Stop:",
            style = {'font-family': 'Segoe UI', 'padding': '0 2em'}),
        dcc.Dropdown(
            id="stop-dropdown",
            options=[],
            value='Rabb Steps',
            style = {'font-family': 'Segoe UI', 'padding': '0 2em'}
        ),
    ], style={'width': '75%'}),
    
    dcc.Graph(id="bar-graph",
        style = {'padding': '0 2em'})
], style={'font-family': 'Segoe UI', 'padding':'0.5em'})

### Updating Slider Options

In [20]:
# Callback to update the stop-selector options and initial value based on the selected route
@app.callback(
    [Output('stop-selector', 'options'),
     Output('stop-selector', 'value')],  # Set the initial value of stop-selector
    Input('route-selector', 'value')
)
def update_stop_selector_options(selected_route):
    # Filter stops based on the selected route
    stops_for_route = df[df['Route'] == selected_route]['Stop'].unique()
    stop_options = [{'label': stop, 'value': stop} for stop in stops_for_route]
    return stop_options, stops_for_route  # Set the initial value

# Callback to update the date-slider options based on the selected route
@app.callback(
    Output('date-slider', 'start_date'),
    Output('date-slider', 'end_date'),
    Input('route-selector', 'value')
)
def update_date_slider(selected_route):
    if selected_route == 'Route A':
        # Define the date range associated with Route A
        start_date = datetime.date(2023, 1, 1)
        end_date = datetime.date(2023, 1, 7)
    elif selected_route == 'Route B':
        # Define the date range associated with Route B
        start_date = datetime.date(2023, 1, 8)
        end_date = datetime.date(2023, 1, 14)
    else:
        # Default date range
        start_date = df['Day'].min()
        end_date = df['Day'].max()
    
    return start_date, end_date

## Entire Stop Graph

In [21]:
# Callback for the Stop bar chart
@app.callback(
    Output('stop-bar-chart', 'figure'),
    [Input('route-selector', 'value'),
     Input('date-slider', 'start_date'),
     Input('date-slider', 'end_date'),
     Input('stop-selector', 'value')]
)
def update_stop_chart(selected_route, start_date, end_date, selected_stops):
    filtered_df = df[(df['Route'] == selected_route) & (df['Stop'].isin(selected_stops))]
    
    avg_stop_data = filtered_df.groupby('Stop')[['Riders On', 'Riders Off']].mean().reset_index()
    
    fig = go.Figure(data=[
        go.Bar(name='Riders On', x=avg_stop_data['Stop'], y=avg_stop_data['Riders On']),
        go.Bar(name='Riders Off', x=avg_stop_data['Stop'], y=avg_stop_data['Riders Off'])
    ])
    
    fig.update_layout(barmode='group', title=f'Average Riders On and Off at Stops for Route {selected_route}')
    return fig


## Average Stop Ridership over Time of Day

### Slider Update Code

In [22]:
@app.callback(
    Output("stop-dropdown", "options"),
    Input("route-selector", "value")
)
def update_stop_options(selected_route):
    if selected_route is None:
        # Handle the case when no route is selected
        return []
    
    stop_options = [{'label': stop, 'value': stop} for stop in df[df['Route'] == selected_route]['Stop'].unique()]
    return stop_options

### Visualization Code

In [23]:
@app.callback(
    Output("bar-graph", "figure"),
    [Input("stop-dropdown", "value"), 
     Input("route-selector", "value"),
     Input("date-slider", "start_date"), 
     Input("date-slider", "end_date")]
)
def update_graph(selected_stop, selected_route, start_date, end_date):
    filtered_data = df[(df['Stop'] == selected_stop) & (df['Route'] == selected_route)
                      & (df['Day'] >= start_date) & (df['Day'] <= end_date)]
    avg_riders = filtered_data.groupby('Scheduled Time')[['Riders On', 'Riders Off']].mean().reset_index()
    
#     fig = make_subplots()
#     fig.add_trace(go.Bar(x=avg_riders['Scheduled Time'], y=avg_riders['Riders On']))
    
    fig = go.Figure(data=[
        go.Bar(x=avg_riders['Scheduled Time'], y=avg_riders['Riders On'], name='Riders On'),
        go.Bar(x=avg_riders['Scheduled Time'], y=avg_riders['Riders Off'], name='Riders Off')
    ])
    
    fig.update_layout(barmode='stack',
                      title=f"Average Riders On/Off for Stop: {selected_stop}, Route: {selected_route}",
                      xaxis_title="Scheduled Time",
                      yaxis_title="Average Riders On/Off",
                      font=dict(family='Segoe UI', size=12, color='black'))
    
    return fig

## Run Application

In [24]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)